In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time
import h5py
import copy

import matplotlib.pyplot as plt
import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import procrustes

import viz
import cameras
import data_utils
import linear_model

import matplotlib.gridspec as gridspec

In [2]:
# モデルのパスを指定して構築
train_dir = '../ckpt'
ckpt = tf.train.get_checkpoint_state(train_dir, latest_filename="checkpoint")

summaries_dir = os.path.join( train_dir, "log" )
model = linear_model.LinearModel(
    1024,
    2,
    True,
    False,
    True,
    64,
    1e-3,
    summaries_dir,
    False,
    dtype=tf.float32)

if ckpt and ckpt.model_checkpoint_path:
    ckpt_name = os.path.basename( ckpt.model_checkpoint_path )






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








In [3]:
with tf.Session() as sess:
    
    # モデルへweightを読み込む
    print("Loading model {0}".format( ckpt_name ))
    model.saver.restore( sess, ckpt.model_checkpoint_path )
    print("Model loaded")
    
    enc_in = np.arange(32).reshape(1,32)
    dec_out = np.arange(48).reshape(1,48)
    dp = 1.0
    
    _, _, poses3d = model.step(sess, enc_in, dec_out, dp, isTraining=False)

Loading model checkpoint-4874200
INFO:tensorflow:Restoring parameters from ../ckpt/checkpoint-4874200
Model loaded


In [4]:
poses3d

array([[-0.0394242 ,  0.02336586,  0.02405731,  0.02954462,  0.12618995,
        -0.01935147,  0.04881043,  0.12481637,  0.04060541,  0.03942417,
        -0.02336448, -0.0240575 ,  0.1481902 ,  0.13176751, -0.03242367,
         0.18603157,  0.13249151, -0.04657107,  0.3833069 ,  0.33933014,
        -0.04174181,  0.32632723,  0.1805447 , -0.03046812,  0.24835709,
         0.21106756, -0.021041  ,  0.2531911 ,  0.19681375, -0.02256853,
         0.2397516 ,  0.21001111, -0.015431  ,  0.19216438,  0.26242778,
        -0.0394512 ,  0.21380308,  0.3658872 ,  0.03567064,  0.20204845,
         0.23013574, -0.03016636,  0.18658516,  0.3752058 , -0.00718565,
         0.1809592 ,  0.3686468 ,  0.04924582]], dtype=float32)

In [5]:
dim_to_ignore_3d = data_utils.get_dim_to_ignore(3)

In [6]:
dim_to_ignore_3d

array([ 0,  1,  2, 12, 13, 14, 15, 16, 17, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 48, 49, 50, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])

In [14]:
data_mean_3d = np.arange(48)
data_std_3d = np.arange(48)

poses3d = data_utils.unNormalizeData( poses3d, data_mean_3d, data_std_3d, dim_to_ignore_3d )

ValueError: shape mismatch: value array of shape (1,48) could not be broadcast to indexing result of shape (30,1)

In [16]:
D = data_mean_3d.shape[0]
dimensions_to_use = np.array([dim for dim in range(D)
                                if dim not in dim_to_ignore_3d])
dimensions_to_use

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])

可視化パート

In [ ]:
fig = plt.figure( figsize=(19.2, 10.8) )
gs1 = gridspec.GridSpec(5, 9) # 5 rows, 9 columns
gs1.update(wspace=-0.00, hspace=0.05) # set the spacing between axes.
plt.axis('off')
subplot_idx, exidx = 1, 1
nsamples = 15

for i in np.arange( nsamples ):

    # Plot 2d pose
#     ax1 = plt.subplot(gs1[subplot_idx-1])
#     p2d = enc_in
#     viz.show2Dpose( p2d, ax1 )
#     ax1.invert_yaxis()

#     # Plot 3d gt
#     ax2 = plt.subplot(gs1[subplot_idx], projection='3d')
#     p3d = dec_out
#     viz.show3Dpose( p3d, ax2 )

    # Plot 3d predictions
    ax3 = plt.subplot(gs1[subplot_idx+1], projection='3d')
    p3d = poses3d
    viz.show3Dpose( p3d, ax3, lcolor="#9b59b6", rcolor="#2ecc71" )

    exidx = exidx + 1
    subplot_idx = subplot_idx + 3

    plt.show()

以下草稿

In [ ]:
with tf.Session() as sess:

    # モデルのパスを指定
    train_dir = '../ckpt'
    ckpt = tf.train.get_checkpoint_state(train_dir, latest_filename="checkpoint")

    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename( ckpt.model_checkpoint_path )

    # .metaからモデルを読み込む
    print("Loading model {0}".format( ckpt_name ))
    saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path + '.meta')
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Model loaded") 
    
    print(sess)